In [2]:
%%capture
!pip install geopandas
!pip install sddk

In [146]:
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)

import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
import time
import sddk
from bs4 import BeautifulSoup as bs
import math

In [4]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

Your ScienceData username (e.g. '123456@au.dk'): 648597@au.dk
Your ScienceData password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [9]:
# import the datasets...
PHI = pd.read_json("https://zenodo.org/record/4972968/files/PHI_v1.0.json?download=1")

# Minimal test

In [ ]:
PHI_sample = PHI.sample(1000, random_state=1)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link
60228,/text/180672?location=1691&patt=&bookid=225&of...,Epigr. tou Oropou,487,"Regions\nCentral Greece (IG VII-IX)\nMegaris, ...",Epigr. tou Oropou\n487,Boiotia — Oropos (Skala) — Amphiareion — 3rd c...,"Dedication. Small base of white marble, broke...",2,1\n,[— — —]εθνη[— — —]\n[— — —] ἀνέθηκε.,...,"[[ἔθνος, ἀνατίθημι]]","[ἔθνος, ἀνατίθημι]",3rd c. BC,"{'or': None, 'not_before': -300, 'not_after': ...",-300.0,-201.0,None,"[range, cent]",None,None
159554,/text/293424?location=1703&patt=&bookid=172&bo...,SEG,32:161,"Regions\nEgypt, Nubia and CyrenaïcaEgypt and N...",SEG\n32:161,Att. — Athens — 402/1 BC,,114,stele I.1\n\n\n\n5\n\ncol. I.6\n\n\n\n10\n\n\n...,Εὐθύμαχ[ο]ς ΑΛ̣Κ̣[— — — c.14 — — — ἐγραμμάτ]-\...,...,"[[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρ...","[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρυ...",402/1 BC,"{'or': None, 'not_before': -402, 'not_after': ...",-402.0,-401.0,None,[range],None,None
209067,/text/344689?location=1665&patt=&bookid=874&of...,"IG XII,6",2:944,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,6\n2:944",Aeg. Islands — Samos Isl. — Prov. unkn. [Samos...,"Epitaph. Plaque of gray limestone, intact at ...",7,\n\n\n\n\n5\n,"vacat 0,03\n[— — — χ]αῖρε\n[— — — —]α̣ς μνή-\n...",...,"[[χαίρω, μνήμη, μήτε, ξαϲ, ρου]]","[χαίρω, μνήμη, μήτε, ξαϲ, ρου]",5th/6th c. AD,"{'or': None, 'not_before': 401, 'not_after': 6...",401.0,600.0,None,"[range, cent, morece]",None,None
149118,/text/282987?location=1617&patt=&bookid=663&of...,IK Rhod. Peraia,220,"Regions\nAsia Minor\nCaria, Rhodian Peraia",IK Rhod. Peraia\n220,"Car., Rhod.Per. — Tymnos (Bozburun) — Hellenis...",,2,1\n,Θ̣αρσύν[ο]ντι\nἥρωι.,...,"[[θαρσύνοντι, ἥρως]]","[θαρσύνοντι, ἥρως]",Hellenistic period,"{'or': None, 'source': 'PeriodO', 'notes': Non...",-330.0,-30.0,None,"[range, period]",None,http://n2t.net/ark:/99152/p03wskd55sw
52953,/text/170921?location=1687&patt=&bookid=190&of...,IK Byzantion,74,Regions\nThrace and the Lower Danube (IG X)\nT...,IK Byzantion\n74,Thrace — Byzantion (Istanbul) — 2nd c. BC — Fı...,,2,1\n,Εὐκλέων\nΜενίσκου.,...,"[[εὐκλεής, μενίσκου]]","[εὐκλεής, μενίσκου]",2nd c. BC,"{'or': None, 'not_before': -200, 'not_after': ...",-200.0,-101.0,None,"[range, cent]",None,None


In [ ]:
# trismegistos api
base_url = "https://www.trismegistos.org/dataservices/texrelations/{0}?source=phi"

In [ ]:
%%time
resps = []
for n in PHI_sample["PHI_ID"][:100]:
    resp_json = requests.get(base_url.format(str(n))).json()
    resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    resps.append(resp_json)

CPU times: user 1.47 s, sys: 107 ms, total: 1.58 s
Wall time: 1min 1s


In [ ]:
PHI_sample_alts = pd.DataFrame.from_records(resps)
PHI_sample_alts.head(10)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,TLA,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida
0,[907493],None,None,None,None,None,None,None,[180672],None,...,None,None,None,None,None,None,None,None,None,None
1,[884657],None,None,None,None,None,None,None,[293424],None,...,None,None,None,None,None,None,None,None,None,None
2,[782973],None,None,None,None,None,None,None,[344689],None,...,None,None,None,None,None,None,None,None,None,None
3,[868178],None,None,None,None,None,None,None,[282987],None,...,None,None,None,None,None,None,None,None,None,None
4,[865289],None,None,None,None,None,None,None,[170921],None,...,None,None,None,None,None,None,None,None,None,None
5,[276850],None,None,[39100774],[EDR115435],None,None,None,[187966],None,...,None,None,None,None,None,None,None,None,None,None
6,[934389],None,None,None,None,None,None,None,[181314],None,...,None,None,None,None,None,None,None,None,None,None
7,[762226],None,None,None,None,None,None,None,[170436],None,...,None,None,None,None,None,None,None,None,None,None
8,[764389],None,None,None,None,None,None,None,[169775],None,...,None,None,None,None,None,None,None,None,None,None
9,[813516],None,None,None,None,None,None,None,[147503],None,...,None,None,None,None,None,None,None,None,None,None


# Main function (using paralel requests)

In [ ]:
def get_inscription_data(phi_id):
    try:
        resp_json = requests.get(base_url.format(str(phi_id))).json()
        resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    except:
        time.sleep(1)
        try:
            resp_json = requests.get(base_url.format(str(phi_id))).json()
            resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
        except:
            resp_json = {}
    return resp_json

# Minimal experiment with paralels

In [ ]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 300, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)

CPU times: user 5.51 s, sys: 539 ms, total: 6.05 s
Wall time: 5.06 s


In [ ]:

pd.DataFrame(all_inscriptions)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,[786251],NaN,NaN,None,NaN,NaN,NaN,NaN,[1],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[786252],NaN,NaN,None,NaN,NaN,NaN,NaN,[2],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[786253],NaN,NaN,None,NaN,NaN,NaN,NaN,[3],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[786254],NaN,NaN,None,NaN,NaN,NaN,NaN,[4],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[786255],NaN,NaN,None,NaN,NaN,NaN,NaN,[5],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,[786544],NaN,NaN,None,NaN,NaN,NaN,NaN,[296],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,[786545],NaN,NaN,None,NaN,NaN,NaN,NaN,[297],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,[786546],NaN,NaN,None,NaN,NaN,NaN,NaN,[298],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,[786547],NaN,NaN,None,NaN,NaN,NaN,NaN,[299],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Test with saving

In [ ]:
list(range(0, len(PHI), step))[-1]

In [ ]:
checkspoints = list(range(0, len(PHI), 1000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139

In [ ]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 3001, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_1000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_2000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_3000.csv"
CPU times: user 56.3 s, sys: 4.65 s, total: 1min
Wall time: 57.2 s


In [ ]:
PHI_TM_1000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(1000)))
PHI_TM_2000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(2000)))
PHI_TM_3000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(3000)))

In [ ]:
PHI_TM_merge = pd.DataFrame()
for df in [PHI_TM_1000, PHI_TM_2000, PHI_TM_3000]:
  PHI_TM_merge = PHI_TM_merge.append(df)

In [ ]:
len(PHI_TM_merge)

# Main extraction

In [ ]:
step=100
checkspoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

all_inscriptions = []
for num in range(0, len(PHI), step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []


[5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000, 145000, 150000, 155000, 160000, 165000, 170000, 175000, 180000, 185000, 190000, 195000, 200000, 205000, 210000, 215000, 218100]
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_5000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_10000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_15000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_20000.csv"
Your <class 'pandas.core.frame.DataFrame'> object ha

# Merge files

In [18]:
test = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(80000)))
test.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (5,7,89,90,108,113,119,120,121) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,['781347'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200264'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['781348'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200265'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['781349'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200266'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['781350'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200267'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,['781351'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200268'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,['781352'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200269'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,['781353'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200270'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,['781354'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200271'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,['781355'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200272'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,['781356'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['200273'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
step = 100
checkpoints = list(range(0, len(PHI), 5000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps

PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(checkpoints[0])))
for num in checkspoints[1:]:
  filename = "SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num))
  PHI_TM_merged = PHI_TM_merged.append(s.read_file(filename))
  print(filename)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (90,94,120,124,128,131,143) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (3,10,90,118,120,128,143) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_10000.csv
SDAM_data/PHI/PHI_TM_15000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (89,90,94,113,123,124,131,143) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_20000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (7,23,53,89,94,113,121,128,131) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


...

SDAM_data/PHI/PHI_TM_205000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (5,7,27,33,91,111,113,119,120,121,128,131,133) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_210000.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (7,11,90,113,123,128) have mixed types.Specify dtype option on import or set low_memory=False.
  import sys


SDAM_data/PHI/PHI_TM_215000.csv
SDAM_data/PHI/PHI_TM_218100.csv


In [20]:
len(PHI_TM_merged)

218162

In [22]:
PHI_TM_merged[100000:100010]

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
4900,['942406'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233130'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4901,['942497'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233131'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4902,['942497'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233132'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4903,['929230'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233133'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4904,['788832'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233134'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4905,['788832'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233135'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4906,['902850'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233136'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4907,['902851'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233137'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4908,['902851'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233138'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4909,['942407'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['233139'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
s.write_file("SDAM_data/PHI/PHI_TM_merged.csv", PHI_TM_merged)

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_merged.csv"


In [8]:
PHI_TM_merged = s.read_file("SDAM_data/PHI/PHI_TM_merged.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,5,7,8,10,11,12,13,20,23,25,27,29,32,33,34,53,54,55,56,66,67,70,89,90,91,94,96,99,105,107,108,111,112,113,116,118,119,120,121,122,123,124,125,126,128,131,132,133,134,135,136,141,143) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [9]:
to_keep = []
for col in PHI_TM_merged.columns:
  length_vals = len([el for el in PHI_TM_merged[col] if "[" in str(el)])
  if length_vals >= 1000:
    to_keep.append(col)
    print(col, length_vals)

TM_ID 218104
EDH 8472
EDCS 23929
EDR 3761
PHI 217863
LUPA 1129
ISic 2465
IG 8147
atticinscriptions 4156
Attalus 2235


In [10]:
PHI_TM_merged = PHI_TM_merged[to_keep]
PHI_TM_merged.head(10)

,TM_ID,EDH,EDCS,EDR,PHI,LUPA,ISic,IG,atticinscriptions,Attalus
0,['786251'],NaN,NaN,NaN,['1'],NaN,NaN,['IG I³ 1'],['1672'],NaN
1,['786252'],NaN,NaN,NaN,['2'],NaN,NaN,['IG I³ 2'],['1707'],NaN
2,['786253'],NaN,NaN,NaN,['3'],NaN,NaN,['IG I³ 3'],['ii-1b'],NaN
3,['786254'],NaN,NaN,NaN,['4'],NaN,NaN,"['IG I³ 4(A)', 'IG I³ 4(B)']",['1692'],NaN
4,['786255'],NaN,NaN,NaN,['5'],NaN,NaN,['IG I³ 5'],['1284'],NaN
5,['786256'],NaN,NaN,NaN,['6'],NaN,NaN,['IG I³ 6'],['6'],NaN
6,['786257'],NaN,NaN,NaN,['7'],NaN,NaN,['IG I³ 7'],['108'],NaN
7,['786258'],NaN,NaN,NaN,['8'],NaN,NaN,['IG I³ 8'],['8'],NaN
8,['786259'],NaN,NaN,NaN,['9'],NaN,NaN,['IG I³ 9'],['9'],NaN
9,['786260'],NaN,NaN,NaN,['10'],NaN,NaN,['IG I³ 10'],['120'],NaN


In [11]:
def try_eval(val):
  if not isinstance(val, list):
    try: 
      val = eval(val)
    except:
      val = None
  return val

In [12]:
for col in PHI_TM_merged.columns:
  PHI_TM_merged[col] = PHI_TM_merged[col].apply(try_eval)

In [13]:
PHI_TM_merged.head(5)

,TM_ID,EDH,EDCS,EDR,PHI,LUPA,ISic,IG,atticinscriptions,Attalus
0,[786251],None,None,None,[1],None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,[2],None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,[3],None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,[4],None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,[5],None,None,[IG I³ 5],[1284],None


In [14]:
for col in PHI_TM_merged.columns:
  if "ID" not in col:
    PHI_TM_merged.rename(columns={col : col + "_ID"}, inplace=True)

In [15]:
PHI_TM_merged.head(5)

,TM_ID,EDH_ID,EDCS_ID,EDR_ID,PHI_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,[786251],None,None,None,[1],None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,[2],None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,[3],None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,[4],None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,[5],None,None,[IG I³ 5],[1284],None


In [16]:
def get_first_id(val):
  try: 
    return int(val[0])
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(get_first_id)

In [17]:
def to_int(val):
  try: 
    return int64(val)
  except: 
    return val

PHI_TM_merged["PHI_ID"] = PHI_TM_merged["PHI_ID"].apply(to_int)

In [18]:
PHI_TM_merged.head(5)

,TM_ID,EDH_ID,EDCS_ID,EDR_ID,PHI_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,[786251],None,None,None,1.0,None,None,[IG I³ 1],[1672],None
1,[786252],None,None,None,2.0,None,None,[IG I³ 2],[1707],None
2,[786253],None,None,None,3.0,None,None,[IG I³ 3],[ii-1b],None
3,[786254],None,None,None,4.0,None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,[786255],None,None,None,5.0,None,None,[IG I³ 5],[1284],None


In [20]:
PHI = PHI.merge(PHI_TM_merged, on="PHI_ID")

In [21]:
s.write_file("SDAM_data/PHI/PHI_TMids.json", PHI)

A file with the same name ("PHI_TMids.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TMids.json"


# Extract TM places

In [5]:
#PHI = s.read_file("SDAM_data/PHI/PHI_TMids.json")

In [33]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,string_pythia,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,clean_text_pythia,sents,sents_N,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link,TM_ID,EDH_ID,EDCS_ID,EDR_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1,ἔδοχσεν τοι δέμοι τ[ὸς ἐ σ]αλαμ[ῖνι κλερόχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,[ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκ...,1,"[[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σα...","[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σαλ...",c. 510-500 a.,"{'or': None, 'not_before': -511, 'not_after': ...",-511.0,-499.0,None,"[range, phase, ca]",None,None,[786251],None,None,None,None,None,[IG I³ 1],[1672],None
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2,[--------9---]ν βολ ---------- [------7--] α ἑ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,"ν βολ ․ α ⋮ ℎεκον σιον, γνοσθε͂ι δὲ ν ἀτεχνος ...",ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,[ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄ...,1,"[[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελ...","[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελε...",c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None,[786252],None,None,None,None,None,[IG I³ 2],[1707],None
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3,[-]αρ[----]ι ἑρακλειο[-----] [-]αρο [-] τιθένα...,αρι ℎερακλειο αρο τιθέναι τὸς ἀέτας τριάκοντα ...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρ ι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,[αρ ι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκ...,1,"[[ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνήρ, ἄγο...","[ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνήρ, ἄγον...",490-480 a.,"{'or': None, 'not_before': -490, 'not_after': ...",-490.0,-480.0,None,[range],None,None,[786253],None,None,None,None,None,[IG I³ 3],[ii-1b],None
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,ἐὰν δέ τις αν ⋮ ἒ φρορὰν ⋮ μὲ πεντέκοντα ⋮ δρα...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,[ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς ...,2,"[[τὶς, φρορὰν, πεντέκοντα, δραχμή, τ, πρᾶχσιν,...","[τὶς, φρορὰν, πεντέκοντα, δραχμή, τ, πρᾶχσιν, ...",485/4 a.,"{'or': None, 'not_before': -485, 'not_after': ...",-485.0,-484.0,None,[range],None,None,[786254],None,None,None,None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5,[ἔδοχσε]ν [ τει βολει] καὶ [τ]οι δέμοι ὅτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,ἔδοχσεν ⋮ τε͂ι βολε͂ι ⋮ καὶ το͂ι δέμοι ⋮ ℎότε ...,ἔδοχσεν τει

In [23]:
TM_IDs = [idlist[0] for idlist in PHI["TM_ID"]]

In [25]:
TM_IDs[:10]

['786251',
 '786252',
 '786253',
 '786254',
 '786255',
 '786256',
 '786257',
 '786258',
 '786259',
 '786260']

In [35]:
len(TM_IDs)

217863

In [34]:
len(list(set(TM_IDs)))

192114

In [53]:
TM_IDs = list(set(TM_IDs))

# Simple test

In [28]:
base_url = "https://www.trismegistos.org/text/" # + 786251

In [51]:
soup = requests.get(base_url + "786251").text

In [48]:
[el.rpartition("/")[2] for el in [a.get("href") for a in soup.find_all("a")] if "place/" in el][0]

'364'

In [49]:
%%time
places_ids = []
for tmid in TM_IDs[:20]:
  soup = bs(requests.get(base_url + tmid).text)
  place_ids = [el.rpartition("/")[2] for el in [a.get("href") for a in soup.find_all("a")] if "place/" in el]
  places_ids.append(place_ids)


CPU times: user 631 ms, sys: 39.3 ms, total: 671 ms
Wall time: 13.6 s


In [50]:
places_ids

[['364'],
 ['373'],
 ['373'],
 ['364'],
 ['10345'],
 ['10345'],
 ['373'],
 ['373'],
 ['373'],
 ['373'],
 ['364'],
 ['373'],
 ['373'],
 ['364'],
 ['364'],
 ['364'],
 ['364'],
 ['373'],
 ['373'],
 ['373']]

In [66]:
def get_tm_places(tmid):
  try:
      soup = bs(requests.get(base_url + tmid).text)
      place_ids = [el.rpartition("/")[2] for el in [a.get("href") for a in soup.find_all("a")] if "place/" in el]
  except:
      time.sleep(1)
      try:
          soup = bs(requests.get(base_url + tmid).text)
          place_ids = [el.rpartition("/")[2] for el in [a.get("href") for a in soup.find_all("a")] if "place/" in el]
      except:
          place_ids = []
  return (tmid, place_ids)

In [142]:
tmid = "786251"
soup = bs(requests.get(base_url + tmid).text) # headers = {"User-Agent": "Safari"}
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Trismegistos encountered an error.</title>
</head>
<body>
<style>

    .float-parent {
        width: 80%;
        margin: 0 auto;
        padding-top: 10%;
    }

    .float-child {
        width: 40%;
        float: left;    }

    img{
        max-width: 400px;
    }
    </style>
<div class="float-parent">
<div class="float-child">
<img src="/img/TM_homeNew.png"/>
</div>
<div class="float-child">
<h1 id="errTitle">403 Forbidden</h1>
<p id="errMsg">The server has refused to fulfil your request.</p>
<p id="errGeneric">Oops we seem to have run into an issue. The incident has been logged and we'll look into it ASAP to make TM greater than ever. If the issue is particularly urgent, please contact us! </p>
<a href="index.php">Take me to the homepage</a>
</div>
</div>
</body>
</html>

In [123]:
get_tm_places("787345")

('787345', [])

In [67]:
%%time
step = 100
with ThreadPoolExecutor(max_workers=step*1.5) as pool:
      currently_parsed = list(pool.map(get_tm_places,TM_IDs[10000:10100]))

CPU times: user 8.36 s, sys: 1.36 s, total: 9.72 s
Wall time: 7.69 s


In [68]:
currently_parsed

[('789754', ['364']),
 ('766802', ['38244']),
 ('636859', ['2058']),
 ('841949', ['56044']),
 ('805024', ['3683']),
 ('127918', ['2394']),
 ('94982', ['1341']),
 ('851668', ['648']),
 ('885197', ['32081']),
 ('771282', ['2057']),
 ('797184', ['373']),
 ('636872', ['2058']),
 ('177918', ['1163']),
 ('120896', ['12098']),
 ('803902', ['2181']),
 ('493799', ['2210']),
 ('103777', ['34']),
 ('178234', ['1495']),
 ('893831', ['2057']),
 ('906877', ['1495']),
 ('920174', ['11774']),
 ('881536', ['364']),
 ('897663', ['12120']),
 ('802400', ['650']),
 ('950489', ['2201']),
 ('899936', ['606']),
 ('801095', ['373']),
 ('803949', ['2181']),
 ('548596', ['14344']),
 ('801830', ['1783']),
 ('891008', ['849']),
 ('926031', ['1600']),
 ('814015', ['543']),
 ('909323', ['2057']),
 ('806766', ['55365']),
 ('850123', ['1688']),
 ('807206', ['1083']),
 ('770531', ['542']),
 ('193448', ['29653']),
 ('843551', ['56209']),
 ('792233', ['373']),
 ('936794', ['648']),
 ('806792', ['54113']),
 ('861511', ['1

In [69]:
step=100
checkpoints = list(range(0, len(TM_IDs), 5000))[1:] # its own range plus last element of steps

all_places = []
for num in range(0, len(TM_IDs), step):
    actual_tmids = TM_IDs[num:num+step]
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_tm_places,actual_tmids))
    all_places.extend(currently_parsed)
    if num in checkpoints:
      s.write_file("SDAM_data/PHI/tmids_to_tmplaces_{0}.csv".format(str(num)), all_places)
      all_places = []

Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_5000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_10000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_15000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_20000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_25000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_30000.csv"
Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_35000.csv"
Your <c

In [71]:
# last elements
all_places = []
for num in range(190100, len(TM_IDs), step):
    actual_tmids = TM_IDs[num:num+step]
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_tm_places,actual_tmids))
    all_places.extend(currently_parsed)

In [73]:
len(all_places)

2014

In [74]:
s.write_file("SDAM_data/PHI/tmids_to_tmplaces_{0}.csv".format(str(192100)), all_places)

Your <class 'list'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_192100.csv"


In [82]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [112]:
step = 100
checkpoints = list(range(5000, len(TM_IDs), 5000)) + [192100] # its own range plus last element of steps

TM_TMgeo_merged = []
for num in checkpoints:
  filename = "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_{0}.csv".format(str(num))
  TM_TMgeo_merged.extend(conf[0].get(filename).json())
  print(filename)

https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_5000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_10000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_15000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_20000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_25000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_30000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_35000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_40000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_45000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_50000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_55000.csv
https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/tmids_to_tmplaces_60000.csv
https://sciencedata.dk/files/

In [113]:
len(TM_TMgeo_merged)

192114

In [138]:
TM_IDs_failed = [el[0] for el in TM_TMgeo_merged if not bool(el[1])]
TM_IDs_failed[:10]

['854335',
 '848794',
 '780139',
 '882086',
 '790912',
 '816219',
 '809617',
 '772568',
 '928015',
 '192022']

In [139]:
TM_TMgeo_dict = dict(TM_TMgeo_merged)
s.write_file("SDAM_data/PHI/TM_TMgeo_dict_1.json", TM_TMgeo_dict)

Your <class 'dict'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/TM_TMgeo_dict_1.json"


In [144]:
datalen = len(TM_IDs_failed)
list(range(0, datalen, 5000))

[0,
 5000,
 10000,
 15000,
 20000,
 25000,
 30000,
 35000,
 40000,
 45000,
 50000,
 55000,
 60000,
 65000,
 70000,
 75000,
 80000,
 85000,
 90000,
 95000,
 100000,
 105000,
 110000,
 115000,
 120000,
 125000,
 130000,
 135000,
 140000]

In [ ]:
step=100
datalen = len(TM_IDs_failed)
checkpoints = list(range(0, datalen, 5000))[1:] + [math.floor(datalen / step) * step]# its own range plus last element of steps

all_places = []
for num in range(0, len(TM_IDs_failed), step):
    actual_tmids = TM_IDs[num:num+step]
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_tm_places,actual_tmids))
    all_places.extend(currently_parsed)
    if num in checkpoints:
      s.write_file("SDAM_data/PHI/tmids_to_tmplaces_{0}_run2.json".format(str(num)), all_places)
      all_places = []

In [116]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,string_pythia,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,clean_text_pythia,sents,sents_N,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link,TM_ID,EDH_ID,EDCS_ID,EDR_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID
0,/text/1?location=1701&patt=&bookid=4&offset=0&...,IG I³,1,Regions\nAttica (IG I-III),IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,,12,1\n\n\n\n5\n\n\n\n\n10\n\n,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,IG-I³.csv,1,ἔδοχσεν τοι δέμοι τ[ὸς ἐ σ]αλαμ[ῖνι κλερόχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκε...,[ἔδοχσεν τοι δέμοι τὸς ἐ σαλαμῖνι κλερόχος οἰκ...,1,"[[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σα...","[ἔδοχσεν, δέμοι, Σαλαμίς, κλερόχος, οἰκεν, Σαλ...",c. 510-500 a.,"{'or': None, 'not_before': -511, 'not_after': ...",-511.0,-499.0,None,"[range, phase, ca]",None,None,[786251],None,None,None,None,None,[IG I³ 1],[1672],None
1,/text/2?location=1701&patt=&bookid=4&offset=0&...,IG I³,2,Regions\nAttica (IG I-III),IG I³\n2,Att. — non-stoich. — c. 500 a.,,14,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,IG-I³.csv,2,[--------9---]ν βολ ---------- [------7--] α ἑ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,"ν βολ ․ α ⋮ ℎεκον σιον, γνοσθε͂ι δὲ ν ἀτεχνος ...",ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄλ...,[ν βολ α ἑκον σιον γνοσθει δὲ ν ἀτεχνος μὲ π ἄ...,1,"[[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελ...","[βολ, ἑκών, σίον, γνοσθει, ἄτεχνος, μεδὲ, κελε...",c. 500 a.,"{'or': None, 'not_before': -505, 'not_after': ...",-505.0,-495.0,None,"[exact, phase, ca]",None,None,[786252],None,None,None,None,None,[IG I³ 2],[1707],None
2,/text/3?location=1701&patt=&bookid=4&offset=0&...,IG I³,3,Regions\nAttica (IG I-III),IG I³\n3,Att. — stoich. 21 — 490-480 a.,,13,1\n\n\n\n5\n\n\n\n\n10\n\n\n,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,IG-I³.csv,3,[-]αρ[----]ι ἑρακλειο[-----] [-]αρο [-] τιθένα...,αρι ℎερακλειο αρο τιθέναι τὸς ἀέτας τριάκοντα ...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,αρ ι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,[αρ ι ἑρακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκ...,1,"[[ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνήρ, ἄγο...","[ἑρακλειο, ἀρόω, τίθημι, ἀθλοθέτης, ἀνήρ, ἄγον...",490-480 a.,"{'or': None, 'not_before': -490, 'not_after': ...",-490.0,-480.0,None,[range],None,None,[786253],None,None,None,None,None,[IG I³ 3],[ii-1b],None
3,/text/4?location=1701&patt=&bookid=4&offset=0&...,IG I³,4,Regions\nAttica (IG I-III),IG I³\n4,Att. — stoich. 38 — 485/4 a.,,56,face A.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,IG-I³.csv,4,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,ἐὰν δέ τις αν ⋮ ἒ φρορὰν ⋮ μὲ πεντέκοντα ⋮ δρα...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,[ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς ...,2,"[[τὶς, φρορὰν, πεντέκοντα, δραχμή, τ, πρᾶχσιν,...","[τὶς, φρορὰν, πεντέκοντα, δραχμή, τ, πρᾶχσιν, ...",485/4 a.,"{'or': None, 'not_before': -485, 'not_after': ...",-485.0,-484.0,None,[range],None,None,[786254],None,None,None,None,None,"[IG I³ 4(A), IG I³ 4(B)]",[1692],None
4,/text/5?location=1701&patt=&bookid=4&offset=0&...,IG I³,5,Regions\nAttica (IG I-III),IG I³\n5,Att. — c. 500 a.,,6,1\n\n\n\n5\n,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,IG-I³.csv,5,[ἔδοχσε]ν [ τει βολει] καὶ [τ]οι δέμοι ὅτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,ἔδοχσεν ⋮ τε͂ι βολε͂ι ⋮ καὶ το͂ι δέμοι ⋮ ℎότε ...,ἔδοχσεν τει

In [118]:
PHI["TMplaces_IDs"] = PHI["TM_ID"].apply(lambda x: TM_TMgeo_dict[str(x[0])])

In [120]:
PHI[1000:1100]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID,string_pythia,clean_text_conservative,clean_text_interpretive_word,clean_text_interpretive_sentence,clean_text_pythia,sents,sents_N,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link,TM_ID,EDH_ID,EDCS_ID,EDR_ID,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMplaces_IDs
1000,/text/1002?location=1701&patt=&bookid=4&offset...,IG I³,887,Regions\nAttica (IG I-III),IG I³\n887,Att. — Athens: Akropolis — stoich. — c. 450-42...,,2,1\n,[–⏕ –⏕ –⏕ –⏕ – —]ιγένος παῖς /\n[–⏕ –⏕ –– χάρι...,IG-I³.csv,1002,[–⏕ –⏕ –⏕ –⏕ – —]ιγένος παῖς /\n[–⏕ –⏕ –– χάρι...,ιγένος παῖς δίδο,ιγένος παῖς χάριν ἀντιδίδο,ιγένος παῖς χάριν ἀντιδίδο.,ιγένος παῖς χάριν ἀντιδίδο.,[ιγένος παῖς χάριν ἀντιδίδο.],1,"[[ιγένος, παῖς, χάρις, ἀντιδίδο]]","[ιγένος, παῖς, χάρις, ἀντιδίδο]",c. 450-425? a.,"{'or': None, 'certainty': '?', 'not_before': -...",-452.0,-423.0,None,"[range, phase, ca]",?,None,[787250],None,None,None,None,None,[IG I³ 887],None,None,[]
1001,/text/1003?location=1701&patt=&bookid=4&offset...,IG I³,888,Regions\nAttica (IG I-III),IG I³\n888,Att. — Athens: Akropolis — c. 440? a. — IG I² ...,,1,1,[{²ὁ δεῖνα}² ἀνέθεκεν] Εὐμελίδο γυνὲ Σφεττόθεν.,IG-I³.csv,1003,[{²ὁ δεῖνα}² ἀνέθεκεν] Εὐμελίδο γυνὲ Σφεττόθεν.,Εὐμελίδο γυνὲ Σφεττόθεν,ὁ δεῖνα ἀνέθεκεν Εὐμελίδο γυνὲ Σφεττόθεν,ὁ δεῖνα ἀνέθεκεν Εὐμελίδο γυνὲ Σφεττόθεν.,ὁ δεῖνα ἀνέθεκεν Εὐμελίδο γυνὲ Σφεττόθεν.,[ὁ δεῖνα ἀνέθεκεν Εὐμελίδο γυνὲ Σφεττόθεν.],1,"[[δεῖνα, ἀνέθεκεν, εὐμελίδο, γυνὲ]]","[δεῖνα, ἀνέθεκεν, εὐμελίδο, γυνὲ]",c. 440? a.,"{'or': None, 'certainty': '?', 'not_before': -...",-445.0,-435.0,None,"[exact, phase, ca]",?,None,[787251],None,None,None,None,None,[IG I³ 888],None,None,[]
1002,/text/1004?location=1701&patt=&bookid=4&offset...,IG I³,889,Regions\nAttica (IG I-III),IG I³\n889,Att. — Athens: Akropolis — c. 440? a. — IG I² ...,,1,1,— — —εὺς Λευκαίο υἱός.,IG-I³.csv,1004,— — —εὺς Λευκαίο υἱός.,εὺς Λευκαίο υἱός,εὺς Λευκαίο υἱός,εὺς Λευκαίο υἱός.,εὺς Λευκαίο υἱός.,[εὺς Λευκαίο υἱός.],1,"[[εὺς, λευκαίο, υἱός]]","[εὺς, λευκαίο, υἱός]",c. 440? a.,"{'or': None, 'certainty': '?', 'not_before': -...",-445.0,-435.0,None,"[exact, phase, ca]",?,None,[787252],None,None,None,None,None,[IG I³ 889],None,None,[]
1003,/text/1005?location=1701&patt=&bookid=4&offset...,IG I³,890,Regions\nAttica (IG I-III),IG I³\n890,Att. — Athens: Akropolis — stoich. — c. 440? a...,,3,1\n\n,Φιλο[․․․․]\nΜελι[τεὺς]\nἀνέθ[εκεν].,IG-I³.csv,1005,Φιλο[․․․․]\nΜελι[τεὺς]\nἀνέθ[εκεν].,Φιλο Μελι ἀνέθ,Φιλο Μελιτεὺς ἀνέθεκεν,Φιλο Μελιτεὺς ἀνέθεκεν.,Φιλο Μελιτεὺς ἀνέθεκεν.,[Φιλο Μελιτεὺς ἀνέθεκεν.],1,"[[φιλο, μελιτεὺς, ἀνέθεκεν]]","[φιλο, μελιτεὺς, ἀνέθεκεν]",c. 440? a.,"{'or': None, 'certainty': '?', 'not_before': -...",-445.0,-435.0,None,"[exact, phase, ca]",?,None,[787253],None,None,None,None,None,[IG I³ 890],None,None,[]
1004,/text/1006?location=1701&patt=&bookid=4&offset...,IG I³,891,Regions\nAttica (IG I-III),IG I³\n891,Att. — Athens: Akropolis — stoich. — c. 440? a...,,2,1\n,Αἰσχύλος ἀνέθε[κεν]\nΠυθέο Παιανιεύ[ς].,IG-I³.csv,1006,Αἰσχύλος ἀνέθε[κεν]\nΠυθέο Παιανιεύ[ς].,Αἰσχύλος ἀνέθε Πυθέο Παιανιεύ,Αἰσχύλος ἀνέθεκεν Πυθέο Παιανιεύς,Αἰσχύλος ἀνέθεκεν Πυθέο Παιανιεύς.,Αἰσχύλος ἀνέθεκεν Πυθέο Παιανιεύς.,[Αἰσχύλος ἀνέθεκεν Πυθέο Παιανιεύς.],1,"[[αἰσχύλος, ἀνέθεκεν, πυθέο, παιανιεύς]]","[αἰσχύλος, ἀνέθεκεν, πυθέο, παιανιεύς]",c. 440? a.,"{'or': None, 'certainty': '?', 'not_before': -...",-445.0,-435.0,None,"[exact, phase, ca]",?,None,[787254],None,None,None,None,None,[IG I³ 891],None,None,[364]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,/text/1097?location=1701&patt=&bookid=4&offset...,IG I³,970,Regions\nAttica (IG I-III),IG I³\n970,Att. — Eleusis — 425-406 a. — IG II² 3090,,5,1\n\n\n\n5,"[Γ]νάθις Τιμοκ[ήδ]ο[ς, Ἀ]ναξανδρίδης Τιμα[γ]όρ...",IG-I³.csv,1097,[γ]νάθις τιμοκ[ήδ]ο[ς ἀ]ναξανδρίδης τιμα[γ]όρο...,νάθις Τιμοκο